In [1]:
import graph_gen_integration
from GraphGen.graph_gen_integration import draw_graph

graph_gen = graph_gen_integration.GraphGen([], keyword_reward=1.1)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.8: Fast Gemma2 patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA GeForce RTX 2060. Max memory: 6.0 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unsloth 2024.8 patched 26 layers with 26 QKV layers, 26 O layers and 26 MLP layers.


In [2]:
triples = graph_gen.generate("""Quiz bowl (quizbowl,[1] scholars' bowl, scholastic bowl, academic bowl, academic team, academic challenge, etc.) is a family of quiz-based competitions that test players on a wide variety of academic subjects. Standardized quiz bowl formats are played by primary school, middle school, high school, and university students throughout North America, Asia, Europe, Australia, and Africa.[2]

Quiz bowl competitions are typically played with a lockout buzzer system[1] between at least two teams, usually consisting of four players each. A moderator reads questions to the players, who try to score points for their team by buzzing first and responding with the correct answer.

Quiz bowl is most commonly played in a toss-up/bonus format, which consists of a series of two different types of questions. Other formats, particularly in local competitions, may deviate from the above rules, with additions like lightning rounds or category choice.[3]

Most forms of modern quiz bowl are modeled after game shows.[4] College Bowl, which was created by Don Reid as a USO activity for U.S. service men during World War II, was an influential early quiz bowl program.[5][6] Also known as "The College Quiz Bowl," it started on radio in 1953 and then aired on national television in the U.S. from 1959 to 1970.[7]

In the first half of the 20th century, many other quiz-bowl-like competitions were also created. Delco Hi-Q began in 1948 as a radio quiz competition sponsored by the Scott Paper Company for high school students in Delaware County, Pennsylvania. It claims to be the oldest continuously running student quiz contest in the United States.[8] The It's Academic televised student quiz show program has been run for high school teams in the Washington, D.C., metropolitan area since 1961 and is recognized by the Guinness Book of World Records as the longest-running quiz program in television history.[9] It's Academic has been spun off in many other U.S. media markets and has inspired many other televised high school competitions.[9]

In 1977, College Bowl was revived as an activity on college campuses in the U.S. by College Bowl Company Inc. (CBCI).[10] In September 1990, the Academic Competition Federation (ACF) was founded as the first major alternative to The College Bowl Company.[11] National Academic Quiz Tournaments (NAQT) was founded in 1996 and currently organizes national competitions at all levels in the United States and supplies tournament questions for grade school and college teams across North America and other parts of the world.[12][13] In 2008, the College Bowl program abruptly ended in the U.S., although the company itself continues to operate the Honda Campus All-Star Challenge (HCASC) for historically black colleges and universities.[14] 
""")

PROCESSING (1/17)
Processing "Quiz bowl (quizbowl,[1] scholars' bowl, scholastic...nd Africa."
Done!
<unused0>Academic_team<unused1>associatedWith<unused2>"Scholars' Bowl"<unused0>Academic_team<unused1>associatedWith<unused2>"Standardized quiz bowl format"<unused0>Academic_team<unused1>associatedWith<unused2>"Academic Challenge"<unused0>Academic_team<unused1>associatedWith<unused2>"Academic Team"<unused0>Academic_team<unused1>associatedWith<unused2>"Academic Bowl"

Academic_team<unused1>associatedWith<unused2>"Scholars' Bowl"
Academic_team<unused1>associatedWith<unused2>"Standardized quiz bowl format"
Academic_team<unused1>associatedWith<unused2>"Academic Challenge"
Academic_team<unused1>associatedWith<unused2>"Academic Team"
Academic_team<unused1>associatedWith<unused2>"Academic Bowl"
DONE
PROCESSING (2/17)
Processing "Standardized quiz bowl formats are played by prima...nd Africa."
Done!
<unused0>North_America<unused1>region<unused2>"Africa"<unused0>North_America<unused1>region<unuse

In [15]:
graph_gen.free_model()

NameError: name 'graph_gen' is not defined

In [14]:
del graph_gen

In [3]:
graph_gen_integration.draw_graph(triples)

network.html


## Check accuracy
### Network

In [1]:
from graph_gen_integration import Triple
import graph_gen_integration
graph_gen = graph_gen_integration.GraphGen([], keyword_reward=1.1)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.8: Fast Gemma2 patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA GeForce RTX 2060. Max memory: 6.0 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unsloth 2024.8 patched 26 layers with 26 QKV layers, 26 O layers and 26 MLP layers.


In [3]:
# load testing dataset
import pandas as pd
import networkx as nx
import re
graph_gen_dataset = pd.read_csv('data/webnlg_test.csv', header=0, delimiter='\t')


def select_k(spectrum, minimum_energy = 0.9):
    running_total = 0.0
    total = sum(spectrum)
    if total == 0.0:
        return len(spectrum)
    for i in range(len(spectrum)):
        running_total += spectrum[i]
        if running_total / total >= minimum_energy:
            return i + 1
    return len(spectrum)


similarities = []
for idx, data in graph_gen_dataset.iterrows():
    if idx >= 30:
        break
    
    # ground truth
    graph1 = nx.DiGraph()

    for _triple in data["Triples"].split("<T>"):
        print(_triple)
        try:
            if _triple == "":
                continue
            split = re.split("<R>|<S>", _triple)
            subject = split[0]
            predicate = split[1]
            _object = split[2]

            graph1.add_edge(subject, _object)

        except Exception as e:
            print(f"NON-STANDARD TRIPLE {_triple} ({e})")
            continue
    
    # Predicted
    graph_gen.reset_model([])
    triples = graph_gen.generate(data["NL"])
    
    graph2 = nx.DiGraph()
    
    for triple in triples:
        graph2.add_edge(triple.subject, triple.object)
    
    # Similarity
    laplacian1 = nx.spectrum.laplacian_spectrum(graph1)
    laplacian2 = nx.spectrum.laplacian_spectrum(graph2)

    k1 = select_k(laplacian1)
    k2 = select_k(laplacian2)
    k = min(k1, k2)

    similarity = sum((laplacian1[:k] - laplacian2[:k])**2)
    print(similarity)
    similarities.append(similarity)



Turn_Me_On_(album)<R>runtime<S>35.1
Turn_Me_On_(album)<R>producer<S>Wharton_Tiers
Turn_Me_On_(album)<R>followedBy<S>Take_It_Off!
PROCESSING (1/1)
Processing "Turn Me On is a 35.1 minute long album produced by...ke it Off."
Done!
<unused0>Wharton_Tiers<unused1>albumTitle<unused2>Take_it_Off<unused0>Wharton_Tiers<unused1>albumTitle<unused2>Turn_Me_On

Wharton_Tiers<unused1>albumTitle<unused2>Take_it_Off
Wharton_Tiers<unused1>albumTitle<unused2>Turn_Me_On
DONE
4.0

Trane<R>location<S>Swords,_Dublin
PROCESSING (1/1)
Processing "The location of Trane is Swords, Dublin....s, Dublin."
Done!
<unused0>Trane<unused1>location<unused2>Swords,_Dublin

Trane<unused1>location<unused2>Swords,_Dublin
DONE
0.0

Ciudad_Ayala<R>populationMetro<S>1777539
Ciudad_Ayala<R>leaderTitle<S>"City Manager"
Ciudad_Ayala<R>type<S>City
Ciudad_Ayala<R>populationDensity<S>1604.0
Ciudad_Ayala<R>governmentType<S>Council-manager_government
Ciudad_Ayala<R>utcOffset<S>−6
Ciudad_Ayala<R>isPartOf<S>Morelos
PROCESSING (1/1)
Pr

In [5]:
import numpy as np
np.average(similarities)

np.float64(5.517595468166681)

ChatGPT

In [23]:
prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Extract the most confident information in the sentence below as much as possible, and express the relationships in RDF Triples that complement the existing RDF triples. Do not use information from common sense.
### Example:
Input:Marriott International, with Bill Marriott as a key leader, is the tenant of AC Hotel Bella Sky in Copenhagen.
Response:<T>AC_Hotel_Bella_Sky_Copenhagen<R>location<S>Copenhagen<T>AC_Hotel_Bella_Sky_Copenhagen<R>tenant<S>Marriott_International<T>Marriott_International<R>keyPerson<S>Bill_Marriott
### Input:
{}
### Response:
{}"""
import openai
openai.api_key = "" # Nuh uh

similarities = []
for idx, data in graph_gen_dataset.iterrows():
    print(idx)
    if idx >= 30:
        break
    
    # ground truth
    graph1 = nx.DiGraph()

    for _triple in data["Triples"].split("<T>"):
        print(_triple)
        try:
            if _triple == "":
                continue
            split = re.split("<R>|<S>", _triple)
            subject = split[0]
            predicate = split[1]
            _object = split[2]

            graph1.add_edge(subject, _object)

        except Exception as e:
            print(f"NON-STANDARD TRIPLE {_triple} ({e})")
            continue
            
    # OpenAI
    
    # print(prompt.format(data["NL"], "").removesuffix("<unused0>"))
    completion = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": prompt.format(data["NL"], "").removesuffix("<unused0>")}
        ]
    )
    # print(completion.choices[0].message.content)
    graph2 = nx.DiGraph()
    for _triple in completion.choices[0].message.content.split("<T>"):
        print(_triple)
        try:
            if _triple == "":
                continue
            split = re.split("<R>|<S>", _triple)
            subject = split[0]
            predicate = split[1]
            _object = split[2]

            graph2.add_edge(subject, _object)

        except Exception as e:
            print(f"NON-STANDARD TRIPLE {_triple} ({e})")
            continue
            
    # Similarity
    laplacian1 = nx.spectrum.laplacian_spectrum(graph1)
    laplacian2 = nx.spectrum.laplacian_spectrum(graph2)

    k1 = select_k(laplacian1)
    k2 = select_k(laplacian2)
    k = min(k1, k2)

    similarity = sum((laplacian1[:k] - laplacian2[:k])**2)
    print(similarity)
    similarities.append(similarity)



0

Turn_Me_On_(album)<R>runtime<S>35.1
Turn_Me_On_(album)<R>producer<S>Wharton_Tiers
Turn_Me_On_(album)<R>followedBy<S>Take_It_Off!

Turn_Me_On<R>duration<S>35.1_minutes

Turn_Me_On<R>producer<S>Wharton_Tiers

Turn_Me_On<R>followedBy<S>Take_it_Off
0.0
1

Trane<R>location<S>Swords,_Dublin

Trane<R>location<S>Dublin
Trane<R>location<S>Swords
1.0
2

Ciudad_Ayala<R>populationMetro<S>1777539
Ciudad_Ayala<R>leaderTitle<S>"City Manager"
Ciudad_Ayala<R>type<S>City
Ciudad_Ayala<R>populationDensity<S>1604.0
Ciudad_Ayala<R>governmentType<S>Council-manager_government
Ciudad_Ayala<R>utcOffset<S>−6
Ciudad_Ayala<R>isPartOf<S>Morelos

Ciudad_Ayala<R>partOf<S>Morelos

Ciudad_Ayala<R>populationDensity<S>1604.0

Ciudad_Ayala<R>population<S>1777539

Ciudad_Ayala<R>utcOffset<S>-6

Ciudad_Ayala<R>governmentType<S>council-manager_government

Ciudad_Ayala<R>leader<S>City_Manager
36.0
3

ALCO_RS-3<R>powerType<S>Diesel-electric_transmission
ALCO_RS-3<R>length<S>17068.8 (millimetres)

ALCO_RS-3<R>length<S>17068.

In [24]:
np.average(similarities)

np.float64(3.566666666666667)